In [1]:

from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    model_name="llama-3.1-8b-instant",
    groq_api_key='API_KEY'
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-platform/job/R-62064")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer II, Platform










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Men

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res= chain_extract.invoke(input={"page_data": page_data})
print(res.content)
type(res.content)

{
  "Software Engineer II, Platform": {
    "role": "Software Engineer II, Platform",
    "experience": {
      "required": "2+ years of relevant experience",
      "degree": "Bachelor's degree in computer science, software engineering or related field"
    },
    "skills": [
      "Java, Python, Golang, or Node.js",
      "AWS services (S3, Route 53, ELB/ALBs, SQS/SNS)",
      "Docker and Kubernetes for containerization and deployment",
      "streaming technologies such as Kafka, Pulsar, Spark Streaming, Flink",
      "SQL or NoSQL databases",
      "continuous integration, unit testing, static analysis, and automated integration tests",
      "Scrum and agile principles"
    ],
    "description": "The candidate plays an integral role in developing and sustaining real-time data streaming systems with dedication and a problem-solving attitude. A robust foundation in AWS cloud infrastructure, programming, and CI/CD practices is advantageous, coupled with a commitment to continuous skil

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res= json_parser.parse(res.content)
json_res

{'Software Engineer II, Platform': {'role': 'Software Engineer II, Platform',
  'experience': {'required': '2+ years of relevant experience',
   'degree': "Bachelor's degree in computer science, software engineering or related field"},
  'skills': ['Java, Python, Golang, or Node.js',
   'AWS services (S3, Route 53, ELB/ALBs, SQS/SNS)',
   'Docker and Kubernetes for containerization and deployment',
   'streaming technologies such as Kafka, Pulsar, Spark Streaming, Flink',
   'SQL or NoSQL databases',
   'continuous integration, unit testing, static analysis, and automated integration tests',
   'Scrum and agile principles'],
  'description': 'The candidate plays an integral role in developing and sustaining real-time data streaming systems with dedication and a problem-solving attitude. A robust foundation in AWS cloud infrastructure, programming, and CI/CD practices is advantageous, coupled with a commitment to continuous skill enhancement.'}}

In [6]:
type(json_res)

dict

In [7]:
import pdfplumber
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain_core.prompts import PromptTemplate


# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        return "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

resume_struct_prompt = PromptTemplate.from_template(
    """
    ### Resume Text:
    {resume_text}

    Extract the following details from the resume text below and format them as a JSON in this structure:

    resume_data = {{
        "name": "<Full Name in string>",
        "education": [<list of education entries in list of strings for each degree>],
        "experience": ["<For each job, provide a single descriptive string combining company, position, duration, location, and key responsibilities/accomplishments>"],
        "projects": [<For each project, provide a single descriptive string summarizing the project title, technologies used, duration, and outcome or objective>],
        "skills": [<list of skills in list of strings>]
    }}
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE): 
    """
)

pdf_path = "Sahithya_ArvetiNagaraju.pdf"
resume_text = extract_text_from_pdf(pdf_path)
chain_extract_resume = resume_struct_prompt | llm
resume_res= chain_extract_resume.invoke(input={"resume_text": resume_text})
print(resume_res.content)
type(resume_res.content)

```json
{
  "name": "Sahithya Arveti Nagaraju",
  "education": [
    "University at Buffalo (GPA 3.97/4.00) Jan. 2024 – May 2025 Master of Science in Data Science Buffalo, New York",
    "Siddaganga Institute of Technology (CGPA 9.44/10.00) Aug. 2017 – Jun. 2021 Bachelor of Science in Information Science and Engineering India"
  ],
  "experience": [
    "Graduate Teaching Assistant at University at Buffalo Jan 2025 - May 2025 Buffalo, NY, United States Collaborated with Professor Khinkis on refining lecture materials for advanced statistical methods and data mining (using R), incorporating real-world case studies that improved student comprehension scores by 15% according to course evaluations.",
    "Software Engineer II at Sabre Aug 2021 – Jan 2024 Bengaluru, India Migrated GetThere services from legacy web servers to Google Cloud Platform using Terraform, Docker, and Kubernetes, improving scalability, optimizing traffic, and cutting infrastructure costs by 22%.",
    "Software Engin

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_resume= json_parser.parse(resume_res.content)
resume_data = json_resume
resume_data

{'name': 'Sahithya Arveti Nagaraju',
 'education': ['University at Buffalo (GPA 3.97/4.00) Jan. 2024 – May 2025 Master of Science in Data Science Buffalo, New York',
  'Siddaganga Institute of Technology (CGPA 9.44/10.00) Aug. 2017 – Jun. 2021 Bachelor of Science in Information Science and Engineering India'],
 'experience': ['Graduate Teaching Assistant at University at Buffalo Jan 2025 - May 2025 Buffalo, NY, United States Collaborated with Professor Khinkis on refining lecture materials for advanced statistical methods and data mining (using R), incorporating real-world case studies that improved student comprehension scores by 15% according to course evaluations.',
  'Software Engineer II at Sabre Aug 2021 – Jan 2024 Bengaluru, India Migrated GetThere services from legacy web servers to Google Cloud Platform using Terraform, Docker, and Kubernetes, improving scalability, optimizing traffic, and cutting infrastructure costs by 22%.',
  'Software Engineer (Intern) at Sabre Jan 2021 –

In [9]:
import uuid
import chromadb
from langchain.prompts import PromptTemplate
from langchain_core.runnables import Runnable
import json

# --- Initialize ChromaDB ---
client = chromadb.PersistentClient(path='vectorstore_resume')
collection = client.get_or_create_collection(name="resume_data")

# --- Ingest each resume section ---
if not collection.count():
    for section, items in resume_data.items():
        if isinstance(items, list):
            for item in items:
                collection.add(
                    documents=[item],
                    metadatas={"section": section},
                    ids=[str(uuid.uuid4())]
                )
        else:
            print(section, items)
            collection.add(
                documents=[items],
                metadatas={"section": section},
                ids=[str(uuid.uuid4())]
            )

collection.get()

{'ids': ['1bdd017b-6166-47da-a9ed-3ec4e32e50f8',
  'ebbdc4c6-cbba-4088-8c42-1ed74182347b',
  '8f565320-ea04-4198-ba14-94a8f8215f27',
  'f68ad19c-34a0-4ba9-bc67-d3497f1b96a4',
  'a8ffe761-6e5b-412e-8d9a-fdd023ab3d25',
  '37e7d312-c1f0-44e4-809d-e62a20c715e3',
  '03e21e85-63fc-472f-b4ee-de496202310a',
  '0f3daee6-5253-4900-8e19-f446393d5cc1',
  'd21a3e1c-7776-4d35-9562-1e48928f0b0f',
  '2da8396a-7f5e-4472-b1f9-cf165a50277a',
  '2444028d-122a-4cdb-8043-b34c16b4b0d7',
  'f1003110-935b-41a5-b0c7-d0fc802db47b',
  '71ee3249-0b80-4ac7-bf78-2028e13c0c00',
  '9eca13d4-d18e-4286-897d-31870959dce7',
  '02ba72de-deca-46ad-b313-dc89285f2d19',
  '837ded3c-ed0e-4511-bada-54bb67e55549',
  '3d09ec01-b44d-4af4-8fa7-29584dff31d1',
  '96324b12-671d-4232-ad31-d06fd856f4df',
  '68c82fe8-32f0-49ee-80a0-8495fedbeff5',
  '6fa672eb-4951-42e8-a792-2645924ff368'],
 'embeddings': None,
 'documents': ['Sahithya Arveti Nagaraju',
  'University at Buffalo (GPA 3.97/4.00) Jan. 2024 – May 2025 Master of Science in Data 

In [62]:
all_docs=collection.get()
all_docs
collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [10]:
job = json_res
skills = job['Software Engineer II, Platform']['skills']
print(skills)

['Java, Python, Golang, or Node.js', 'AWS services (S3, Route 53, ELB/ALBs, SQS/SNS)', 'Docker and Kubernetes for containerization and deployment', 'streaming technologies such as Kafka, Pulsar, Spark Streaming, Flink', 'SQL or NoSQL databases', 'continuous integration, unit testing, static analysis, and automated integration tests', 'Scrum and agile principles']


In [11]:
links = collection.query(query_texts=skills, n_results=2).get('metadatas', [])
links

[[{'section': 'skills'}, {'section': 'skills'}],
 [{'section': 'skills'}, {'section': 'projects'}],
 [{'section': 'skills'}, {'section': 'projects'}],
 [{'section': 'skills'}, {'section': 'skills'}],
 [{'section': 'skills'}, {'section': 'skills'}],
 [{'section': 'experience'}, {'section': 'skills'}],
 [{'section': 'experience'}, {'section': 'education'}]]

In [12]:
skills = job['Software Engineer II, Platform']['skills']
query = skills
results = collection.query(query_texts=query, n_results=6)
relevant_info = [m['section'] + ": " + d for d, m in zip(results['documents'][0], results['metadatas'][0])]
resume_context = "\n".join(relevant_info)

# --- Step 3: Cover Letter Prompt Template ---
prompt_coverletter = PromptTemplate.from_template(
    """
    ### JOB TITLE:
    {job_title}

    ### JOB DESCRIPTION:
    {job_description}
    
    ### RESUME CONTEXT:
    {resume_context}
    
    ### INSTRUCTION:
    Write a professional cover letter tailored for the above job title and description.
    Match and highlight relevant experience and skills from the resume context.
    Keep the tone formal and enthusiastic.
    Avoid generic clichés. Focus on measurable impacts, technologies used, and specific experiences.
    Address it to "Hiring Manager".
    Sign off with "Sincerely, [Your Name]".

    ### COVER LETTER (NO PREAMBLE):
    """
)

# --- Step 4: Invoke the Chain ---
chain_coverletter = prompt_coverletter | llm  # Assumes `llm` is your language model
res = chain_coverletter.invoke({
    "job_title": job['Software Engineer II, Platform']['role'],
    "job_description": job['Software Engineer II, Platform']['description'],
    "resume_context": resume_context
})

print(res.content)

Dear Hiring Manager,

I am excited to apply for the Software Engineer II, Platform position at your esteemed organization. With a strong foundation in developing and sustaining real-time data streaming systems, I am confident that my skills and experience make me an ideal fit for this role.

As a seasoned software engineer with a proven track record of delivering high-impact projects, I am well-equipped to leverage my expertise in AWS cloud infrastructure, programming, and CI/CD practices to drive innovation and growth. My experience in migrating GetThere services from legacy web servers to Google Cloud Platform using Terraform, Docker, and Kubernetes has not only improved scalability but also optimized traffic and reduced infrastructure costs by 22%. I am eager to bring my expertise to a new challenge and contribute to the development and sustenance of real-time data streaming systems.

In my previous role at Sabre, I successfully created a ServiceNow API-based application using Pytho

In [8]:
!pip install bs4


   ---------------------------------------- 0/3 [soupsieve]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [bs4]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!pip install pdfplumber

  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 43.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 38.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 58.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 60.9 MB/s eta 0:00:00
Using cached pycparser-2.22-py3-none-any.whl (117 kB)

   ---------------------------------------- 0/7 [pypdfium2]
   ---------------------------------------- 0/7 [pypdfium2]
   ---------------------------------------- 0/7 [pypdfium2]
   ---------------------------------------- 0/7 [pypdfium2]
   ---------------------------------------- 0/7 [pypd


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
